# Using langchain's chatollama and langchain's streaming
## *beautifulsoup4, requests, ollama library(pypi), langchain-ollama, [ollama](https://ollama.com/) and [llama3.2](https://ollama.com/library/llama3.2)*
### have ollama running - ollama serve/ ollama run llama3.2

This code has the same functionality as [task1](https://github.com/krisx2/BlackDeep-jobTasks/blob/main/task1.ipynb) but I'm using langchain to stream and adjust temperature. Due to the streaming and the use of langchain this code will perform better on large articles


In [ ]:
!pip install requests beautifulsoup4 langchain-ollama

In [ ]:
import requests
from bs4 import BeautifulSoup
from langchain_ollama import ChatOllama

custom_article_url = "https://www.bbc.com/news/technology-65855333" # No need for user input()
sentence_count = 4           # for longer or shorter summary

def fetch_article(article_url):
    try:
        response = requests.get(article_url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, 'html.parser')

        # Extract all paragraph (<p>) tags
        fetched_paragraphs = soup.find_all('p')
        article_content = " ".join([p.get_text(strip=True) for p in fetched_paragraphs])

        # Check if any paragraphs were found
        if not article_content.strip():
            raise Exception("No content found in <p> tags.Try another website")

        return article_content

    except requests.exceptions.RequestException as e:
        raise Exception(f"Couldn't fetch the article. Try another one with a valid link. Error {e}")
#starting the llm
llm = ChatOllama(
    model = "llama3.2",
    temperature = 0.1, # Low temperature for concise summary
    num_predict = 512,
)

fetched_article = fetch_article(custom_article_url)
messages = [
    ("system", "Add a appropriate title for the summary.Provide the title and summary with no other outputs and no other special characters."),
    ("user", f"Summarize this article in {sentence_count} short, concise sentences: {fetched_article}"),
]
#using streaming for better performance and larger sites
stream = llm.stream(messages)
full = []
for chunk in stream:
    full.append(chunk.content)
full_content = ''.join(full)
print(full_content)